In [4]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets
!pip install transformers
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
ERROR: Ignored the following yanked versions: 2.0.0
ERROR: Could not find a version that satisfies the requirement torchaudio==0.10.0+cu113 (from versions: 0.11.0, 0.11.0+cu113, 0.12.0, 0.12.0+cu113, 0.12.1, 0.12.1+cu113, 0.13.0, 0.13.1, 2.0.1, 2.0.2, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1)
ERROR: No matching distribution found for torchaudio==0.10.0+cu113


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
!pip install evaluate
from datasets import load_dataset, Audio
from evaluate import load


In [7]:
common_voice_train = load_dataset("mozilla-foundation/common_voice_17_0", "ne-NP", split="validated+other")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
# Load the WER metric
wer_metric = load("wer")

In [9]:
# This cell basically splits the dataset into three subsets. 80% goes to the training set, 10% goes to the validation set and 10% goes to the test set
from datasets import load_dataset

# Define proportions for training, validation, and test sets
train_ratio = 0.8  # 80% for training
validation_ratio = 0.1  # 10% for validation
test_ratio = 0.1  # 10% for testing
common_voice_train = load_dataset("mozilla-foundation/common_voice_17_0", "ne-NP", split="validated+other")
# Calculate sizes based on ratios
num_samples = len(common_voice_train)
print(f'Original: {num_samples}')
train_size = int(train_ratio * num_samples)
validation_size = int(validation_ratio * num_samples)
test_size = num_samples - train_size - validation_size

# Split the dataset
train_dataset = common_voice_train.select(range(0, train_size))
common_voice_dev = common_voice_train.select(range(train_size, train_size + validation_size))
common_voice_test = common_voice_train.select(range(train_size + validation_size, train_size + validation_size + test_size))

common_voice_train = train_dataset

# Print the sizes of each dataset
print(f"Training dataset size: {len(common_voice_train)} samples")
print(f"Validation dataset size: {len(common_voice_dev)} samples")
print(f"Test dataset size: {len(common_voice_test)} samples")

Original: 1337
Training dataset size: 1069 samples
Validation dataset size: 133 samples
Test dataset size: 135 samples


In [10]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voiec_dev = common_voice_dev.remove_columns(['accent', 'age', 'client_id', 'down_votes', 'gender', 'locale', 'segment', 'up_votes'])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [11]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [12]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_dev = common_voice_dev.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

In [13]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("iamtangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")
processor = Wav2Vec2Processor.from_pretrained("iamtangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

## Below is for Common Voice


In [14]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_dev = common_voice_dev.cast_column("audio", Audio(sampling_rate=16_000))

In [15]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

स्कुल जाने बेलामा ।


In [16]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]['sentence'])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: भोक लागिसक्यो ।
Input array shape: (40320,)
Sampling rate: 16000


In [17]:
common_voice_test[0]['audio']['sampling_rate']

16000

In [18]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [19]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_dev = common_voice_dev.map(prepare_dataset, remove_columns=common_voice_dev.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

In [20]:
# Define proportions for training, validation, and test sets
train_ratio = 0.8  # 80% for training
validation_ratio = 0.1  # 10% for validation
test_ratio = 0.1  # 10% for testing
common_voice_train_evaluation = load_dataset("mozilla-foundation/common_voice_17_0", "ne-NP", split="validated+other")
# Calculate sizes based on ratios
num_samples = len(common_voice_train_evaluation)
print(f'Original: {num_samples}')
train_size = int(train_ratio * num_samples)
validation_size = int(validation_ratio * num_samples)
test_size = num_samples - train_size - validation_size

# Split the dataset
train_dataset = common_voice_train_evaluation.select(range(0, train_size))
common_voice_dev_evaluation = common_voice_train_evaluation.select(range(train_size, train_size + validation_size))
common_voice_test_evaluation = common_voice_train_evaluation.select(range(train_size + validation_size, train_size + validation_size + test_size))

common_voice_train_evaluation = train_dataset

# Print the sizes of each dataset
print(f"Training dataset size: {len(common_voice_train_evaluation)} samples")
print(f"Validation dataset size: {len(common_voice_dev_evaluation)} samples")
print(f"Test dataset size: {len(common_voice_test_evaluation)} samples")

Original: 1337
Training dataset size: 1069 samples
Validation dataset size: 133 samples
Test dataset size: 135 samples


In [21]:
common_voice_train_evaluation = common_voice_train_evaluation.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test_evaluation = common_voice_test_evaluation.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_dev_evaluation = common_voice_dev_evaluation.cast_column("audio", Audio(sampling_rate=16_000))

In [22]:
import torch
import random


In [23]:

random_int = random.randint(0, len(common_voice_dev)-1)
input_dict = processor(common_voice_dev[random_int]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


## Comparison with other's model

In [117]:
# model = AutoModelForCTC.from_pretrained("spktsagar/wav2vec2-large-xls-r-300m-nepali-openslr")
# processor = Wav2Vec2Processor.from_pretrained("spktsagar/wav2vec2-large-xls-r-300m-nepali-openslr")
model = AutoModelForCTC.from_pretrained("Strange18/wav2vec2-nepali-asr")
processor = Wav2Vec2Processor.from_pretrained("Strange18/wav2vec2-nepali-asr")
model_tang = AutoModelForCTC.from_pretrained("iamtangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")
processor_tang = Wav2Vec2Processor.from_pretrained("iamtangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [118]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC
model_cv =  Wav2Vec2ForCTC.from_pretrained('/content/drive/MyDrive/Nepali_ASR/results/checkpoint-800')
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("/content/drive/MyDrive/Nepali_ASR/results/checkpoint-800", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor_cv = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import IPython.display as ipd
random_int = random.randint(0, len(common_voice_dev_evaluation)-1)
# random_int = 800
# For Common Voice
input_dict_cv = processor_cv(common_voice_dev[random_int]["input_values"], return_tensors="pt", padding=True)
logits_cv = model_cv(input_dict_cv.input_values).logits

# For OpenSLR
input_dict_open_slr = processor_tang(common_voice_dev[random_int]['input_values'], return_tensors='pt', padding=True)
logits_open_slr = model_tang(input_dict_open_slr.input_values).logits

#For Sagar
input_dict_sagar = processor(common_voice_dev[random_int]["input_values"], return_tensors="pt", padding=True)
logits_sagar = model(input_dict_sagar.input_values).logits

#For Sagar
pred_ids_sagar = torch.argmax(logits_sagar, dim=-1)[0]

#For common Voice
pred_ids_cv = torch.argmax(logits_cv, dim=-1)[0]

#For OpenSLR
pred_ids_open_slr = torch.argmax(logits_open_slr, dim=-1)[0]

print("Prediction:")
print("Prediction From Sagar's model: ")
print(processor.decode(pred_ids_sagar))
print("Prediction From our Common Voice model: ")
print(processor_cv.decode(pred_ids_cv))
print("Prediction from our OpenSLR model: ")
print(processor_tang.decode(pred_ids_open_slr))

# Ignore the immediate code below
promptInput = processor_tang.decode(pred_ids_open_slr)


print("\nReference:")
print(common_voice_dev_evaluation[random_int]["sentence"].lower())
ipd.Audio(data=common_voice_dev_evaluation[random_int]['audio']['array'], autoplay=True, rate=16000)


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


## Gemini

In [82]:
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.4/725.4 kB 12.8 MB/s eta 0:00:00


In [84]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [90]:
model_gemini = genai.GenerativeModel('gemini-pro')

In [115]:
# promptInput = '''
# बेयारड र स्टिन ए अफ्रिकी अमेरिकी नागरिक अगुवाथिउनले उन हजे सुरुमा उक हजार नौ सय चालीस र क हजार नौ सय पचासको तशकमा फेलोसीप अफरिकन सिरिय ससंस्थान मार्फत रपछि नागरिक अधिकार आन्दोलनमा्फकउ उत्तर अमेरिकामा सबैका लागि सम्मानअधिकास्थापना का निम्ति खाम गरे
# '''
response = model_gemini.generate_content("Correct Grammar of the following text. It is in Nepali language. The text should be in a clean and readable format without any punctuation. There should be no line breaks. Also, no commas and punctuation symbols. The text is:" + promptInput)
print(response.text)

घाँकी हेरी हाड निल्ने


### From my model

In [ ]:
model = AutoModelForCTC.from_pretrained("iamtangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")
processor = Wav2Vec2Processor.from_pretrained("iamtangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")

In [ ]:
import IPython.display as ipd
random_int = random.randint(0, len(common_voice_train)-1)
random_int = 69
input_dict = processor(common_voice_train[random_int]["input_values"], return_tensors="pt", padding=True)
logits = model(input_dict.input_values).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_train_evaluation[random_int]["sentence"].lower())
ipd.Audio(data=common_voice_train_evaluation[random_int]['audio']['array'], autoplay=True, rate=16000)


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Prediction:
मैले माछा पोखरीतिरछु

Reference:
म अहिले माछापोखरी तिर छु ।


In [ ]:
print('Hello World! Thank You!!')

Hello World! Thank You!!


# Inference From a File

In [28]:
from IPython.display import Audio
from scipy.io import wavfile
import numpy as np

##### Sagar Reference Model

In [40]:

# model_sagar = Wav2Vec2ForCTC.from_pretrained('spktsagar/wav2vec2-large-xls-r-300m-nepali-openslr')
# processor_sagar = Wav2Vec2Processor.from_pretrained('spktsagar/wav2vec2-large-xls-r-300m-nepali-openslr')
model_sagar = Wav2Vec2ForCTC.from_pretrained('anish-shilpakar/wav2vec2-nepali')
processor_sagar = Wav2Vec2Processor.from_pretrained('anish-shilpakar/wav2vec2-nepali')

config.json:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at anish-shilpakar/wav2vec2-nepali were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at anish-shilpakar/wav2vec2-nepali and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN thi

preprocessor_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/278 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/814 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from transformers import Wav2Vec2ForCTC

# tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("/content/drive/MyDrive/Nepali_ASR/results/checkpoint-800", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
model = Wav2Vec2ForCTC.from_pretrained('/content/drive/MyDrive/Nepali_ASR/results/checkpoint-800/')

In [ ]:
from transformers import Wav2Vec2Tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("/content/drive/MyDrive/Nepali_ASR/results/checkpoint-800", unk_token='[UNK]', pad_token='[PAD]', word_delimiter_token='|')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:720: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(


In [69]:
file_name = '/content/drive/MyDrive/Nepali_ASR/record_out-7.wav'

In [70]:
!pip install pydub

In [72]:
from pydub import AudioSegment

# Load MP3 file
audio = AudioSegment.from_mp3(file_name)

# Export as WAV
wav_file = "/content/drive/MyDrive/Nepali_ASR/test-file-wav.wav"
audio.export(wav_file, format="wav")


<_io.BufferedRandom name='/content/drive/MyDrive/Nepali_ASR/test-file-wav.wav'>

In [73]:
Audio(wav_file)

In [74]:
data = wavfile.read(wav_file)
framerate = data[0]
sounddata = data[1]
time = np.arange(0,len(sounddata))/framerate
print('Sample rate:',framerate,'Hz')
print('Total time:',len(sounddata)/framerate,'s')


Sample rate: 48000 Hz
Total time: 5.94 s


In [75]:
import soundfile as sf
import librosa

In [76]:
input_audio, _ = librosa.load(wav_file,
                              sr=16000)

In [77]:
input_dict_cv = processor_cv(input_audio, return_tensors="pt", padding=True)
logits_cv = model_cv(input_dict_cv.input_values).logits

# For real Sagar. Other sagar is stranger's thapathali's model
input_dict_rsg = processor_sagar(input_audio, return_tensors="pt", padding=True)
logits_dict_rsg = model_sagar(input_dict_rsg.input_values).logits

# For OpenSLR
input_dict_open_slr = processor_tang(input_audio, return_tensors='pt', padding=True)
logits_open_slr = model_tang(input_dict_open_slr.input_values).logits

#For Sagar
input_dict_sagar = processor(input_audio, return_tensors="pt", padding=True)
logits_sagar = model(input_dict_sagar.input_values).logits

#For Sagar
pred_ids_sagar = torch.argmax(logits_sagar, dim=-1)[0]

#For common Voice
pred_ids_cv = torch.argmax(logits_cv, dim=-1)[0]

#For OpenSLR
pred_ids_open_slr = torch.argmax(logits_open_slr, dim=-1)[0]

# For real Sagar
pred_ids_rsg = torch.argmax(logits_dict_rsg, dim=-1)[0]

print("Prediction:")
print("Prediction From real Sagar's model: ")
print(processor_sagar.decode(pred_ids_rsg))
print("Prediction From Sagar's model: ")
print(processor.decode(pred_ids_sagar))
print("Prediction From our Common Voice model: ")
print(processor_cv.decode(pred_ids_cv))
print("Prediction from our OpenSLR model: ")
print(processor_tang.decode(pred_ids_open_slr))


# print("\nReference:")
# print(common_voice_dev_evaluation[random_int]["sentence"].lower())
ipd.Audio(data=input_audio, autoplay=True, rate=16000)

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Prediction:
Prediction From real Sagar's model: 
तिनी कल्पना हो वास्तवेकतामा छैनु
Prediction From Sagar's model: 
तिमी कल्बना होौ बास्ट बिगतामा छैनु
Prediction From our Common Voice model: 
तिमी कलबना हौ वास्त विकतामा छैनौ ।
Prediction from our OpenSLR model: 
तिनी कलपना हो वास्तविकतामा छैनौ


## For longer version clips

In [37]:
import librosa
import torch

def chunk_audio(audio, chunk_duration, overlap_duration, sample_rate=16000):
    # Convert duration from seconds to samples
    chunk_samples = int(chunk_duration * sample_rate)
    overlap_samples = int(overlap_duration * sample_rate)

    chunks = []
    # Iterate through the audio in steps, subtracting the overlap to ensure continuity
    for start in range(0, len(audio) - overlap_samples, chunk_samples - overlap_samples):
        end = start + chunk_samples
        chunk = audio[start:end]
        chunks.append(chunk)
    return chunks


In [38]:
# Parameters for chunking
chunk_duration = 5  # Duration of each chunk in seconds
overlap_duration = 0.5  # Overlap duration in seconds

# Load your 30-second audio
# Assuming input_audio is a NumPy array or a list of audio samples
# input_audio, _ = librosa.load("path/to/30-second-audio.wav", sr=16000)

# Create chunks of the audio
audio_chunks = chunk_audio(input_audio, chunk_duration, overlap_duration)

# Store predictions
predictions_rsg = []
predictions_sagar = []
predictions_cv = []
predictions_open_slr = []

# Perform inference on each chunk
for chunk in audio_chunks:
    # Prepare the input for each model
    input_dict_cv = processor_cv(chunk, return_tensors="pt", padding=True)
    logits_cv = model_cv(input_dict_cv.input_values).logits

    input_dict_rsg = processor_sagar(chunk, return_tensors="pt", padding=True)
    logits_dict_rsg = model_sagar(input_dict_rsg.input_values).logits

    input_dict_open_slr = processor_tang(chunk, return_tensors='pt', padding=True)
    logits_open_slr = model_tang(input_dict_open_slr.input_values).logits

    input_dict_sagar = processor(chunk, return_tensors="pt", padding=True)
    logits_sagar = model(input_dict_sagar.input_values).logits

    # Decode predictions
    pred_ids_sagar = torch.argmax(logits_sagar, dim=-1)[0]
    pred_ids_cv = torch.argmax(logits_cv, dim=-1)[0]
    pred_ids_open_slr = torch.argmax(logits_open_slr, dim=-1)[0]
    pred_ids_rsg = torch.argmax(logits_dict_rsg, dim=-1)[0]

    predictions_rsg.append(processor_sagar.decode(pred_ids_rsg))
    predictions_sagar.append(processor.decode(pred_ids_sagar))
    predictions_cv.append(processor_cv.decode(pred_ids_cv))
    predictions_open_slr.append(processor_tang.decode(pred_ids_open_slr))

# Combine predictions
full_prediction_rsg = " ".join(predictions_rsg)
full_prediction_sagar = " ".join(predictions_sagar)
full_prediction_cv = " ".join(predictions_cv)
full_prediction_open_slr = " ".join(predictions_open_slr)

# Print combined predictions
print("Prediction:")
print("Prediction From real Sagar's model: ")
print(full_prediction_rsg)
print("Prediction From Sagar's model: ")
print(full_prediction_sagar)
print("Prediction From our Common Voice model: ")
print(full_prediction_cv)
print("Prediction from our OpenSLR model: ")
print(full_prediction_open_slr)

# Play the original 30-second audio
ipd.Audio(data=input_audio, autoplay=True, rate=16000)


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_ra

Prediction:
Prediction From real Sagar's model: 
बेयार्ड रस्टिन ए अफ्रिकी अमेरिकी नागरिक अगुवा थिएउनले सुरुमा सन् १९४० र १९५० को दशकमा फेलोसिप अफ रिकन सिलिएसन संस्थार मार्फत रपछि नागरिक अधिकार आन्दोलनमार्फत उत्तर अमेरिकामा सबैका लागि सम्मानअधिकास्थापना कानिम्ति घाम गरे
Prediction From Sagar's model: 
बेयाड रस्टिन एक स्टिन एकअफ्रिकी अमेरिकी नागरिक अगुवा थिएउनले स उनलाई सुरुमा सन् उन्नास स्य चालिस र उन्नाइस सय पचासको दसकमयन भयनफेलोसिप अफभिल्म रिकनसिलिय सन्संस्थान स्थानमार्फत र पछि नागरित अधिकार आन्दोलनमार्फत उत् उत्त र अमेरिकामा सबैका लागि सम्मान अधिकार स्थापना ाकानिम्ती काम गरे
Prediction From our Common Voice model: 
बियार्ड रस्टिन ए स्टिन एक अफ्री की अमेरीकी नागरीक अगुवा थिए हउन्लेश उनले सुरुमाशङ उन्नै सेचालिच र उन्नै सेपचासको दसकमा । मा फेलोशीप अफ पिन् रीकण सिलिए शण संस्था । स्थाँ मार्फत रपछी नागरीट अधिगार अन्दोलन मार्फत उ उत्तरमेरीका मा सबै का लागि सम्मान अधिकार स्था पना । काननिम्ती काम गरे ।
Prediction from our OpenSLR model: 
बेयारड र स्टिन ए अफ्रिकी अमेरिकी नागरिक अगुवाथिउनले उन 

In [ ]:
# input_values = tokenizer(input_audio, return_tensors="pt", padding=True).input_values
input_values = tokenizer(input_audio, return_tensors='pt').input_values
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)[0]

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
print(f'Prediction:\n{transcription}')

Prediction:
म अहिले मैजार र्जेर बोति मोती राखेको छु।
